In [1]:
import seaborn as sns
import metapack as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 

%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()


In [2]:
pkg = mp.jupyter.open_package()
#pkg = mp.jupyter.open_source_package()
pkg

In [3]:
dia = pkg.reference('rasp_diabetes').dataframe()
tracts = pkg.reference('rasp_tracts').dataframe()

tracts['raceeth'] = tracts.raceeth.replace({'aian':'other', 'many': 'other','nhopi':'other'})

In [4]:
dia.raceeth.value_counts()

hisp       28
other      28
black      28
asian      28
nhwhite    28
Name: raceeth, dtype: int64

In [5]:
tracts.raceeth.value_counts()

other      148208
nhwhite     37052
all         37052
black       37052
asian       37052
hisp        37052
white       37052
Name: raceeth, dtype: int64

In [6]:
dia18 = dia[dia.age_min >= 18]
tracts18 = tracts[tracts.age_min >= 18]

In [7]:

dia_merge = pd.DataFrame(
{
    'raceeth': dia18.raceeth,
    'age_range': dia18.apply(lambda r: '{:02d}-{:03d}'.format(r.age_min, r.age_max), axis=1),
    'sex': dia18.sex.apply(lambda v: 'male' if v == 1 else 'female' ),
    'pov':  dia18.pov.apply(lambda v: 'below' if v == 1 else 'above' ),
    'prob': dia18.prob
})

dia_merge.head()

,raceeth,age_range,sex,pov,prob
0,nhwhite,18-024,male,below,0.042474
1,asian,18-024,male,below,0.029804
2,black,18-024,male,below,0.066237
3,hisp,18-024,male,below,0.072074
4,other,18-024,male,below,0.054269


In [8]:
#
tracts_merge = tracts18[tracts18.raceeth != 'white'].copy()
tracts_merge.head()


,geoid,col_name,value_est,value_margin,sex,raceeth,age_range,pov,age_min,age_max,overlapping
37061,14000US06073000100,B17001B_010,0,12,male,black,18-024,below,18,24,0
37062,14000US06073000100,B17001B_011,0,12,male,black,25-034,below,25,34,0
37063,14000US06073000100,B17001B_012,0,12,male,black,35-044,below,35,44,0
37064,14000US06073000100,B17001B_013,0,12,male,black,45-054,below,45,54,0
37065,14000US06073000100,B17001B_014,0,12,male,black,55-064,below,55,64,0


In [9]:
tracts_merge.head()

,geoid,col_name,value_est,value_margin,sex,raceeth,age_range,pov,age_min,age_max,overlapping
37061,14000US06073000100,B17001B_010,0,12,male,black,18-024,below,18,24,0
37062,14000US06073000100,B17001B_011,0,12,male,black,25-034,below,25,34,0
37063,14000US06073000100,B17001B_012,0,12,male,black,35-044,below,35,44,0
37064,14000US06073000100,B17001B_013,0,12,male,black,45-054,below,45,54,0
37065,14000US06073000100,B17001B_014,0,12,male,black,55-064,below,55,64,0


In [10]:
tracts[tracts.overlapping == 0].value_est.sum()

3452837

In [11]:
m = tracts_merge.merge(dia_merge, on=['raceeth','age_range','sex','pov'], how='left')


In [12]:
# For 2017 data, this should be about 3337685 ( whole pop) * 78.2%% (% 18 or older ) == 2610069
s = m[m.overlapping== 0].value_est.sum()
s, s-2610069, (s-2610069)/2610069

(2643923, 33854, 0.012970538326764541)

In [32]:
t = m[m.overlapping==0].copy() 

adjust = 0.091/0.1114
t['adj_prob'] = t.prob * adjust


d =(t.prob*t.value_est).sum()
d2 = (t.adj_prob*t.value_est).sum()
d, d/s, d2, d2/s

(294786.61927791283,
 0.11149591696804817,
 240804.1503975769,
 0.09107835228090111)

In [34]:
t = m[(m.overlapping==0) & (m.raceeth=='black')].copy() 
t['adj_prob'] = t.prob * adjust

d =(t.prob*t.value_est).sum()
d2 = (t.adj_prob*t.value_est).sum()
d, d/s, d2/s

(15136.370366683024, 0.0057249664103996315, 0.0046765883603803095)